In [1]:
import pandas as pd
df = pd.read_csv("/Users/jerry/Downloads/new_retail_data.csv")

In [2]:
print(df.columns.tolist())


['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address', 'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income', 'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases', 'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand', 'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method', 'Order_Status', 'Ratings', 'products']


In [3]:
total_rows = len(df)
print("Total rows:", total_rows)

Total rows: 302010


In [4]:
print(df.columns.tolist())


['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address', 'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income', 'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases', 'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand', 'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method', 'Order_Status', 'Ratings', 'products']


In [5]:
missing_count = df.isnull().sum()
missing_pct = (missing_count / len(df) * 100).round(2)

summary = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct
}).sort_values(by='Missing %', ascending=False)

print(summary)


                  Missing Count  Missing %
Name                        382       0.13
Year                        350       0.12
Date                        359       0.12
Phone                       362       0.12
Total_Amount                350       0.12
Amount                      357       0.12
Total_Purchases             361       0.12
Time                        350       0.12
Shipping_Method             337       0.11
Transaction_ID              333       0.11
Zipcode                     340       0.11
Email                       347       0.11
Address                     315       0.10
Payment_Method              297       0.10
Income                      290       0.10
Customer_ID                 308       0.10
Gender                      317       0.10
Country                     271       0.09
Month                       273       0.09
State                       281       0.09
Product_Category            283       0.09
Product_Brand               281       0.09
City       

In [6]:
df = df.dropna(axis=0)

In [7]:
total_rows = len(df)
print("Total rows:", total_rows)

Total rows: 293911


In [8]:
df['Customer_ID'] = df['Customer_ID'].astype(str)
df['Transaction_ID'] = df['Transaction_ID'].astype(str)


In [9]:
purchases_per_customer = (
    df.dropna(subset=['Customer_ID'])
      .groupby('Customer_ID')['Transaction_ID']
      .nunique()
)


In [51]:
avg_purchases_per_customer = purchases_per_customer.mean()
repeat_rate = (purchases_per_customer >= 2).mean()  # share of customers with ≥2 purchases

print(f"Avg purchases per customer: {avg_purchases_per_customer:.3f}")
print(f"Repeat purchase rate: {repeat_rate:.2%}")

Avg purchases per customer: 3.374
Repeat purchase rate: 86.72%


In [10]:
# Payment method preference (% by order count)
payment_pref = (
    df['Payment_Method']
    .dropna()
    .astype(str)
    .str.strip()
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
)

print("Payment method preference (% of orders):")
print(payment_pref)


Payment method preference (% of orders):
Payment_Method
Credit Card    29.87
Debit Card     25.43
Cash           24.47
PayPal         20.23
Name: proportion, dtype: float64


In [13]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Ensure Amount is numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Ensure Total_Amount is numeric if it exists
if 'Total_Amount' in df.columns:
    df['Total_Amount'] = pd.to_numeric(df['Total_Amount'], errors='coerce')


In [14]:
# Set reference date as the day after the last purchase
today = df['Date'].max() + pd.Timedelta(days=1)

rfm = df.groupby('Customer_ID').agg({
    'Date': lambda x: (today - x.max()).days,  # Recency: days since last purchase
    'Transaction_ID': 'nunique',              # Frequency: number of purchases
    'Amount': 'sum'                            # Monetary: total spend
}).reset_index()

rfm.rename(columns={'Date': 'Recency', 'Transaction_ID': 'Frequency', 'Amount': 'Monetary'}, inplace=True)

# Assign scores (1 to 5) for R, F, and M
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1]).astype(int)  # lower Recency → higher score
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# Total RFM score
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].sum(axis=1)

# Segment customers based on RFM score
def segment(row):
    if row['RFM_Score'] >= 13:
        return 'High-Value'
    elif row['RFM_Score'] >= 9:
        return 'Mid-Value'
    else:
        return 'Low-Value'

rfm['Segment'] = rfm.apply(segment, axis=1)

print("\n=== RFM Analysis (Top 10) ===")
print(rfm.head(10))

# Save RFM results
rfm.to_csv("RFM_results.csv", index=False)



=== RFM Analysis (Top 10) ===
  Customer_ID  Recency  Frequency     Monetary  R_Score  F_Score  M_Score  \
0     10000.0      103          4   940.297743        2        3        4   
1     10001.0      105          5  1517.921543        2        4        5   
2     10002.0       95          5  1117.164676        3        4        4   
3     10003.0      228          2   266.301835        1        1        1   
4     10004.0       31          2   460.702696        4        1        2   
5     10005.0      246          1   439.021139        1        1        2   
6     10006.0       16          4  1022.336464        5        3        4   
7     10007.0       34          5  1090.912878        4        4        4   
8     10008.0        9          5  1667.611565        5        4        5   
9     10009.0       58          3   338.720709        3        2        1   

   RFM_Score     Segment  
0          9   Mid-Value  
1         11   Mid-Value  
2         11   Mid-Value  
3          3 

In [15]:
avg_order_value = df.groupby('Customer_ID').apply(lambda x: x['Amount'].sum() / x['Transaction_ID'].nunique())

# Calculate purchase frequency per year
purchase_span_days = df.groupby('Customer_ID')['Date'].agg(['min','max'])
purchase_span_days['days'] = (purchase_span_days['max'] - purchase_span_days['min']).dt.days + 1
purchase_span_days['purchase_count'] = df.groupby('Customer_ID')['Transaction_ID'].nunique()

purchase_span_days['purchase_frequency_per_year'] = purchase_span_days['purchase_count'] / (purchase_span_days['days'] / 365)

# Expected lifespan (years)
expected_lifespan_years = 3

# Calculate CLV
clv = pd.DataFrame({
    'Customer_ID': avg_order_value.index,
    'Average_Order_Value': avg_order_value.values,
    'Purchase_Frequency_per_Year': purchase_span_days['purchase_frequency_per_year'].values
})
clv['CLV'] = clv['Average_Order_Value'] * clv['Purchase_Frequency_per_Year'] * expected_lifespan_years

# CLV segmentation
clv['CLV_Segment'] = pd.qcut(clv['CLV'], 3, labels=['Low', 'Medium', 'High'])

print("\n=== CLV Analysis (Top 10) ===")
print(clv.head(10))

# Save CLV results
clv.to_csv("CLV_results.csv", index=False)

# ===================== Step 6: Merge RFM & CLV and Identify Target Customers =====================
rfm = pd.read_csv("RFM_results.csv")
clv = pd.read_csv("CLV_results.csv")

merged = pd.merge(rfm, clv, on="Customer_ID", how="inner")

/var/folders/5l/qls4phs14lg3v_4wscl92_4w0000gn/T/ipykernel_67092/3130578664.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_order_value = df.groupby('Customer_ID').apply(lambda x: x['Amount'].sum() / x['Transaction_ID'].nunique())



=== CLV Analysis (Top 10) ===
  Customer_ID  Average_Order_Value  Purchase_Frequency_per_Year  \
0     10000.0           235.074436                     7.724868   
1     10001.0           303.584309                     7.358871   
2     10002.0           223.432935                     8.220721   
3     10003.0           133.150918                     9.480519   
4     10004.0           230.351348                     6.293103   
5     10005.0           439.021139                   365.000000   
6     10006.0           255.584116                     6.666667   
7     10007.0           218.182576                     6.809701   
8     10008.0           333.522313                     8.075221   
9     10009.0           112.906903                    10.631068   

             CLV CLV_Segment  
0    5447.756762      Medium  
1    6702.113264      Medium  
2    5510.339278      Medium  
3    3787.019606         Low  
4    4348.874586         Low  
5  480728.147533        High  
6    5111.6823

In [55]:
# Select customers who are both High-Value in RFM and High in CLV
target_customers = merged[
    (merged['Segment'] == 'High-Value') &
    (merged['CLV_Segment'] == 'High')
]

print(f"Number of target customers: {len(target_customers)}")
print("\n=== Sample Target Customers (Top 10) ===")
print(target_customers[['Customer_ID', 'RFM_Score', 'CLV', 'Recency', 'Frequency', 'Monetary']].head(10))


交集客户数量: 2949

=== 样例客户名单（前10条） ===
     Customer_ID  RFM_Score           CLV  Recency  Frequency     Monetary
66       10069.0         14  11202.670738       29          8  2127.995903
188      10196.0         13   8333.433963       72          7  2085.261101
195      10204.0         15   8480.939035       16          9  2687.566982
232      10242.0         13   9121.147177       64          7  2024.144990
248      10258.0         14   8772.831213       29          6  1554.273293
268      10281.0         13  10710.440006       79          8  2347.493700
333      10350.0         13  11764.978793       95          6  2256.297303
349      10367.0         14   9135.756334       51          7  1902.239675
353      10371.0         14   8308.775257        2          5  1570.699980
361      10379.0         13  13431.091599       47          5  1827.609724


In [ ]:
# Save target customer list for marketing
target_customers.to_csv("Target_Customers.csv", index=False)